# Pre-Processing Pipeline Stage

## Import Dependencies

In [1]:
import pandas  as pd
import numpy   as np
import seaborn as sb
import time    as ti

from pyspark.ml.classification import LogisticRegression
from pyspark.ml.linalg         import Vectors
from pyspark.ml.feature        import OneHotEncoderEstimator, StringIndexer, VectorAssembler, Imputer, StandardScaler
from pyspark.ml                import Pipeline

from pyspark.sql               import SparkSession, SQLContext
from pyspark.sql.types         import StructType, StructField, StringType, FloatType, BooleanType, IntegerType, DoubleType, BinaryType
from pyspark.sql.functions     import countDistinct, col

from os.path                   import exists

## Setup Spark Application

In [2]:
ss = SparkSession.builder\
     .config('spark.executor.memory',       '4G')\
     .config('spark.driver.memory',        '40G')\
     .config('spark.driver.maxResultSize', '10G')\
     .getOrCreate()
sc = ss.sparkContext
sq = SQLContext(sc)

## Read Training Data

In [3]:
%%time

if  not exists('../data/train.parquet.original'):

    ds = StructType([StructField(f'ctr'    ,  FloatType(), True)                      ] + \
                    [StructField(f'i{f:02}',  FloatType(), True) for f in range(1, 14)] + \
                    [StructField(f's{f:02}', StringType(), True) for f in range(1, 27)])

    df = sq.read.format('csv')\
                .options(header = 'true', delimiter = '\t')\
                .schema(ds)\
                .load('../data/train.txt')

    df.write.parquet('../data/train.parquet.original')

df = ss.read.parquet('../data/train.parquet.original')
tf = df.sample(fraction = 0.01, seed = 2019)

num_columns = [f for f in df.columns if 's' in f]
cat_columns = [f for f in df.columns if 's' in f]
cat_indexes = [f'{f}_index' for f in cat_columns]

CPU times: user 4.32 ms, sys: 647 µs, total: 4.96 ms
Wall time: 2.26 s


## Split Dev and Test; SaveOut

## Index Categorical Features

In [4]:
%%time

if  not exists('../data/train.parquet.indexed'):

    stages   = [StringIndexer(inputCol = f, outputCol= f'{f}_index').setHandleInvalid('keep') for f in cat_columns]
    pipeline = Pipeline(stages = stages)
    model    = pipeline.fit(df)
    df       = model.transform(df)

    """
    for c in cat_indexes:
        df = df.withColumn(c, col(c).cast('float'))
    
    df = df.select(['ctr'] + num_columns + cat_indexes)
    """
    df.write.parquet('../data/train.parquet.indexed')
    
df = ss.read.parquet('../data/train.parquet.indexed')
tf = df.sample(fraction = 0.01, seed = 2019)

Py4JJavaError: An error occurred while calling o1674.parquet.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:198)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:159)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:104)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:102)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:122)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:131)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:127)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeQuery$1.apply(SparkPlan.scala:155)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScop

In [ ]:
xf = df

In [ ]:
for c in [f'{f}_index' for f in cat_features]:
    df = df.withColumn(c, col(c).cast('float'))

In [ ]:
xf.describe()

In [ ]:
df.write.parquet('../data/train.parquet')

In [ ]:
df = ss.read.parquet('../data/train.parquet')

In [ ]:
cat_distinct = {}

for f in cat_features:
    s  = ti.time()
    cat_distinct[f] = df.agg(countDistinct(f)).collect()[0][0]
    print( f'{f} : {cat_distinct[f]:>8} : {ti.time() - s:.3f}' )

print( f'sum : {sum(cat_distinct.values()):>8}' )

In [ ]:
cat_distinct = {}

for f in cat_features:
    s  = ti.time()
    cat_distinct[f] = df.agg(countDistinct(f)).collect()[0][0]
    print( f'{f} : {cat_distinct[f]:>8} : {ti.time() - s:.3f}' )

print( f'sum : {sum(cat_distinct.values()):>8}' )

In [ ]:
%%time
imputer = Imputer(inputCols = num_features, outputCols = num_features)
model   = imputer.fit(df)
xf      = model.transform(df)
xf.describe(num_features).toPandas().T

In [ ]:
%%time
num_assembler = VectorAssembler(inputCols = num_features, outputCol = 'num_features')
cat_assembler = VectorAssembler(inputCols = cat_features, outputCol = 'cat_features')
xf            = num_assembler.transform(xf)
#xf            = cat_assembler.transform(xf)

xf.describe(num_features).toPandas().T

In [ ]:
%%time
scaler = StandardScaler(inputCol = 'num_features', outputCol = 'num_features_scaled', withStd = True, withMean = False)
model  = scaler.fit(xf)
xf_sd  = model.transform(xf)

xf_sd.describe().toPandas().T

In [ ]:
%%time
xf_sd.select('num_features_scaled').head( 10 )

In [ ]:
%%time
xf_sd.select(num_features).head( 10 )

In [ ]:
stages  = []
indexes = []

for cat in cat_features :
    cat_index = cat + '_index'
    stages   += [StringIndexer(inputCol = cat, outputCol= cat_index).setHandleInvalid('keep')]
    indexes.append(cat_index)

stages += [StringIndexer(inputCol = 'ctr', outputCol = 'label')]
stages += [VectorAssembler(inputCols = indexes + num_features, outputCol ='features')]

process = Pipeline(stages = stages)

In [ ]:
model   = process.fit(df)

In [ ]:
%%time
xf = model.transform(tf)